In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
import os
print(os.listdir("../input"))
from fastai.imports import *
# Any results you write to the current directory are saved as output.

['Data_Dictionary.xlsx', 'train-mahindra', 'sample_submission_dlc0jkw', 'test_mahindra']


### Read the data

In [2]:
train = pd.read_csv('../input/train-mahindra/train.csv')
test = pd.read_csv('../input/test_mahindra/test.csv')
submit = pd.read_csv('../input/sample_submission_dlc0jkw/sample_submission.csv')

### Helper functions for feature engineering

In [3]:
## To display all the data 
## used from fastai
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

In [4]:
## add columns relevant in a date column
## used from fastai
def add_datepart(df, fldname, drop=True, time=False):
    "Helper function that adds columns relevant to a date."
    fld = df[fldname]
    fld_dtype = fld.dtype
    if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        fld_dtype = np.datetime64

    if not np.issubdtype(fld_dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if time: attr = attr + ['Hour', 'Minute', 'Second']
    for n in attr: df[targ_pre + n] = getattr(fld.dt, n.lower())
    df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
    if drop: df.drop(fldname, axis=1, inplace=True)

In [5]:
## to download csv file from the kernel
from IPython.display import HTML
def create_download_link(title = "Download CSV file", filename = "data.csv"):  
    html = '<a href={filename}>{title}</a>'
    html = html.format(title=title,filename=filename)
    return HTML(html)

In [6]:
## helper function to fix missing values
## creates an extra column indicating if that row is missing from the data or not
## taken from  fastai
def fix_missing(df, col, name, na_dict):
    if is_numeric_dtype(col):
        if pd.isnull(col).sum() or (name in na_dict):
            df[name+'_na'] = pd.isnull(col)
            filler = na_dict[name] if name in na_dict else col.median()
            df[name] = col.fillna(filler)
            na_dict[name] = filler
    return na_dict

### sneak-peak into the data

In [7]:
train.shape, test.shape, train.shape[0] + test.shape[0], submit.shape

((341424, 24), (146765, 23), 488189, (146765, 2))

In [8]:
display_all(train.head())

,reservation_id,booking_date,checkin_date,checkout_date,channel_code,main_product_code,numberofadults,numberofchildren,persontravellingid,resort_region_code,resort_type_code,room_type_booked_code,roomnights,season_holidayed_code,state_code_residence,state_code_resort,total_pax,member_age_buckets,booking_type_code,memberid,cluster_code,reservationstatusid_code,resort_id,amount_spent_per_room_night_scaled
0,07659f3758d8aee27f5a7e2887adeacb67021cb95ada1b...,05/04/18,05/04/18,06/04/18,3,1,2,0,46,3,3,3,1,2.0,7.0,3,3,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,F,C,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,7.706428
1,03930f033646d073462b35d411616323597715ac4fc398...,23/01/15,11/04/15,16/04/15,1,1,2,0,46,3,3,4,5,2.0,7.0,5,2,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,F,A,39fa9ec190eee7b6f4dff1100d6343e10918d044c75eac...,6.662563
2,d145a32920e6587ad95bfe299d80c0affa268220535aaf...,28/01/15,01/02/15,05/02/15,1,1,2,0,47,1,5,4,4,2.0,7.0,1,2,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,E,A,535fa30d7e25dd8a49f1536779734ec8286108d115da50...,7.871602
3,cfd77f44811ed62f25a220b53324cdbafc662a4c9e5f04...,02/05/15,11/06/15,16/06/15,1,1,2,2,46,2,2,3,5,2.0,7.0,2,2,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,D,A,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,5.344943
4,937cff9e4dcfc2459620153dfc8b9962ac22bea67dfb29...,02/09/15,14/12/15,19/12/15,1,1,2,0,46,2,2,4,5,2.0,7.0,2,2,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,D,A,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,7.059346


In [9]:
test.head()

,reservation_id,booking_date,checkin_date,checkout_date,channel_code,main_product_code,numberofadults,numberofchildren,persontravellingid,resort_region_code,resort_type_code,room_type_booked_code,roomnights,season_holidayed_code,state_code_residence,state_code_resort,total_pax,member_age_buckets,booking_type_code,memberid,cluster_code,reservationstatusid_code,resort_id
0,7dae1ce6bc8f69481328f2be5c4943077dad5598b5f66d...,05/04/18,05/04/18,06/04/18,3,2,2,0,45,3,3,4,1,2.0,2.0,3,3,H,1,2114944930dcc42ce5b9b50ae965cf8a9c04e46be63d84...,A,C,9f14025af0065b30e47e23ebb3b491d39ae8ed17d33739...
1,fe0d4e444e1818436c88f72f1cf800536c2f785e59baeb...,24/05/15,23/06/15,27/06/15,1,2,4,0,45,1,1,4,8,2.0,2.0,4,2,H,1,2114944930dcc42ce5b9b50ae965cf8a9c04e46be63d84...,F,A,e7f6c011776e8db7cd330b54174fd76f7d0216b612387a...
2,540bd4285ad8168e8388f84ee74a82cd4f97dc0a404d3e...,21/07/15,07/08/15,10/08/15,3,2,3,0,45,2,2,2,3,4.0,2.0,2,3,H,1,2114944930dcc42ce5b9b50ae965cf8a9c04e46be63d84...,D,A,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...
3,09593c907ae262e46f655b4db9e14f54a19eadcfdd2679...,31/07/15,22/10/15,25/10/15,1,2,3,0,45,1,1,2,3,2.0,2.0,6,3,H,1,2114944930dcc42ce5b9b50ae965cf8a9c04e46be63d84...,F,A,49d180ecf56132819571bf39d9b7b342522a2ac6d23c14...
4,f4c50caac68051faf37551d70bb17eebef2a20e2244cb1...,01/07/16,01/10/16,05/10/16,1,2,4,0,45,1,2,3,8,2.0,2.0,11,2,H,1,2114944930dcc42ce5b9b50ae965cf8a9c04e46be63d84...,F,A,e29c9c180c6279b0b02abd6a1801c7c04082cf486ec027...


In [10]:
submit.head()

,reservation_id,amount_spent_per_room_night_scaled
0,7dae1ce6bc8f69481328f2be5c4943077dad5598b5f66d...,8
1,fe0d4e444e1818436c88f72f1cf800536c2f785e59baeb...,8
2,540bd4285ad8168e8388f84ee74a82cd4f97dc0a404d3e...,8
3,09593c907ae262e46f655b4db9e14f54a19eadcfdd2679...,8
4,f4c50caac68051faf37551d70bb17eebef2a20e2244cb1...,8


In [11]:
## lets check the datatype of the columns
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 341424 entries, 0 to 341423
Data columns (total 24 columns):
reservation_id                        341424 non-null object
booking_date                          341424 non-null object
checkin_date                          341424 non-null object
checkout_date                         341424 non-null object
channel_code                          341424 non-null int64
main_product_code                     341424 non-null int64
numberofadults                        341424 non-null int64
numberofchildren                      341424 non-null int64
persontravellingid                    341424 non-null int64
resort_region_code                    341424 non-null int64
resort_type_code                      341424 non-null int64
room_type_booked_code                 341424 non-null int64
roomnights                            341424 non-null int64
season_holidayed_code                 341310 non-null float64
state_code_residence                  33666

In [12]:
## check for missing values in a column
train.isnull().sum(), test.isnull().sum()

(reservation_id                           0
 booking_date                             0
 checkin_date                             0
 checkout_date                            0
 channel_code                             0
 main_product_code                        0
 numberofadults                           0
 numberofchildren                         0
 persontravellingid                       0
 resort_region_code                       0
 resort_type_code                         0
 room_type_booked_code                    0
 roomnights                               0
 season_holidayed_code                  114
 state_code_residence                  4764
 state_code_resort                        0
 total_pax                                0
 member_age_buckets                       0
 booking_type_code                        0
 memberid                                 0
 cluster_code                             0
 reservationstatusid_code                 0
 resort_id                      

In [13]:
## let's introduce target column in test as well for easier feature engineering
#y = train.amount_spent_per_room_night_scaled
#X = train.drop(['amount_spent_per_room_night_scaled'], axis = 1)
test['amount_spent_per_room_night_scaled'] = 0.00

In [14]:
## give label to data based on train/test and concat both data set
train['type'] = 'train'
test['type'] = 'test'
data = pd.concat([train, test], axis = 0, ignore_index= True)  

 ### Feature engineering

In [15]:
## let's fix the missing values first we noticed in 2 columns
fix_missing(data, data['season_holidayed_code'], 'season_holidayed_code', {})
fix_missing(data, data['state_code_residence'], 'state_code_residence', {})

{'state_code_residence': 8.0}

In [16]:
## creating few columns like total people stayed (adults + children), total adult night, total people night
data['total_people'] = data['numberofadults'] + data['numberofchildren']
data['total_adult_night'] = data['numberofadults'] * data['roomnights']
data['total_people_night'] = data['total_people'] * data['roomnights']

In [17]:
## convert data columns to date format
# data['booking_date'] = pd.to_datetime(data['booking_date'], format= '%d/%m/%y')
# data['checkin_date'] = pd.to_datetime(data['checkin_date'], format= '%d/%m/%y')
# data['checkout_date'] = pd.to_datetime(data['checkout_date'], format= '%d/%m/%y')

## create advance booking days = checkin date - booking date
data['days_advance']=(pd.to_datetime(data['checkin_date'],format= '%d/%m/%y')-pd.to_datetime(data['booking_date'],format= '%d/%m/%y')).dt.days

## actual stays based on checkin and checkout = checkout date - checkin date
data['actual_stay']=(pd.to_datetime(data['checkout_date'],format= '%d/%m/%y')-pd.to_datetime(data['checkin_date'],format= '%d/%m/%y')).dt.days

In [18]:
## lets see the statistics
data.describe()

,channel_code,main_product_code,numberofadults,numberofchildren,persontravellingid,resort_region_code,resort_type_code,room_type_booked_code,roomnights,season_holidayed_code,state_code_residence,state_code_resort,total_pax,booking_type_code,amount_spent_per_room_night_scaled,total_people,total_adult_night,total_people_night,days_advance,actual_stay
count,488189.000000,488189.000000,488189.000000,488189.000000,488189.000000,488189.000000,488189.000000,488189.000000,488189.000000,488189.000000,488189.000000,488189.000000,488189.000000,488189.000000,488189.000000,488189.000000,488189.000000,488189.000000,488189.000000,488189.000000
mean,1.902587,2.207459,3.275522,0.362573,209.955489,1.773303,2.194687,2.889850,3.735949,2.676134,7.269420,4.759472,3.191893,1.123952,5.399685,3.638095,14.750271,16.280447,46.119677,2.466323
std,0.915112,0.821989,1.764458,0.758078,864.891408,0.751550,1.549417,0.804194,2.480057,0.884552,4.684542,3.103048,1.166638,0.329527,3.655736,1.888372,26.392213,27.796234,38.693097,1.094641
min,1.000000,1.000000,0.000000,0.000000,45.000000,1.000000,0.000000,1.000000,-45.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,-180.000000,-315.000000,-2219.000000,1.000000
25%,1.000000,2.000000,2.000000,0.000000,45.000000,1.000000,1.000000,2.000000,2.000000,2.000000,4.000000,2.000000,2.000000,1.000000,0.000000,2.000000,6.000000,6.000000,15.000000,2.000000
50%,2.000000,2.000000,3.000000,0.000000,45.000000,2.000000,2.000000,3.000000,3.000000,3.000000,8.000000,5.000000,3.000000,1.000000,7.311203,3.000000,8.000000,9.000000,33.000000,2.000000
75%,3.000000,3.000000,4.000000,0.000000,45.000000,2.000000,3.000000,3.000000,4.000000,3.000000,8.000000,7.000000,4.000000,1.000000,8.168065,4.000000,16.000000,16.000000,82.000000,3.000000
max,3.000000,7.000000,32.000000,13.000000,4995.000000,3.000000,7.000000,6.000000,80.000000,4.000000,38.000000,13.000000,24.000000,2.000000,10.816652,34.000000,1536.000000,1536.000000,177.000000,26.000000


In [19]:
## room night has a minimum value which is negative. lets see all the rows
data[data['roomnights'] < 0]

,reservation_id,booking_date,checkin_date,checkout_date,channel_code,main_product_code,numberofadults,numberofchildren,persontravellingid,resort_region_code,resort_type_code,room_type_booked_code,roomnights,season_holidayed_code,state_code_residence,state_code_resort,total_pax,member_age_buckets,booking_type_code,memberid,cluster_code,reservationstatusid_code,resort_id,amount_spent_per_room_night_scaled,type,season_holidayed_code_na,state_code_residence_na,total_people,total_adult_night,total_people_night,days_advance,actual_stay
36008,8cf7476b7111e0f969ef00d582f8a0833794239ebd3206...,30/12/14,22/03/15,29/03/15,1,2,4,3,45,1,1,2,-45,3.0,8.0,6,4,D,1,a3af543aae775ab3cac3f789f74d5c0a70e6c0ecacb0db...,F,A,b17ef6d19c7a5b1ee83b907c595526dcb1eb06db8227d6...,7.779262,train,False,False,7,-180,-315,82,7


In [20]:
## its just a row.  replace the values manually based on previus calculation  
data['roomnights'] = data['roomnights'].replace(-45, 7)
data['total_adult_night'] = data['total_adult_night'].replace(-180, 28)
data['total_people_night'] = data['total_people_night'].replace(-315, 49)

In [21]:
## we also see that days advance is negative for some roows. lets see them
data[data['days_advance'] < 0]

,reservation_id,booking_date,checkin_date,checkout_date,channel_code,main_product_code,numberofadults,numberofchildren,persontravellingid,resort_region_code,resort_type_code,room_type_booked_code,roomnights,season_holidayed_code,state_code_residence,state_code_resort,total_pax,member_age_buckets,booking_type_code,memberid,cluster_code,reservationstatusid_code,resort_id,amount_spent_per_room_night_scaled,type,season_holidayed_code_na,state_code_residence_na,total_people,total_adult_night,total_people_night,days_advance,actual_stay
15,821a5e5313029b1bbec6ec738a1b34d6d14c82fca97c47...,05/04/18,08/03/12,11/03/12,3,1,2,0,45,1,1,2,1,3.0,8.0,6,3,H,1,7b68caeb153ac83aeab0b9a679709c558d57a6c9d2bdf6...,F,C,49d180ecf56132819571bf39d9b7b342522a2ac6d23c14...,3.218876,train,False,False,2,2,2,-2219,3
34,a80d2fbea1195021307a13a44642855c60f46145dcf29c...,31/03/18,12/03/12,18/03/12,3,2,3,0,45,1,1,3,2,2.0,5.0,1,3,D,2,d4eee99d5c3238cdda52479fb34125e15208d394af34e7...,E,C,48449a14a4ff7d79bb7a1b6f3d488eba397c36ef25634c...,7.029715,train,False,False,3,6,6,-2210,6
60,51fe7242f55c79aacb499f957c746445ccb51de3c8d206...,05/04/18,24/03/12,25/03/12,3,2,2,0,45,3,3,3,1,2.0,6.0,3,4,I,1,a24896d1582d8b6c0de997d8b7435962e33580ba80839d...,A,C,f5ca38f748a1d6eaf726b8a42fb575c3c71f1864a81433...,9.142010,train,False,False,2,2,2,-2203,1
79,548605b68da37dd5c8555a37650910ce9f110e793b4619...,05/04/18,21/03/12,25/03/12,1,3,3,1,45,3,3,2,6,2.0,4.0,5,3,E,1,3bb1aa01c771c2217fe08bae162569a3177da7d09ae6be...,F,A,7902699be42c8a8e46fbbb4501726517e86b22c56a189f...,7.252902,train,False,False,4,18,24,-2206,4
81,3ac8796577c899cd0177b6f9638515e4feab55d8a72b8b...,05/04/18,23/03/12,26/03/12,1,3,4,0,45,1,1,3,4,2.0,2.0,1,3,E,1,fa856d4dde8f88ed2855a74e2468cedd802ee253434a6c...,E,C,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,7.229536,train,False,False,4,16,16,-2204,3
113,4d73a3304416c4bff78b47c6b307a07568d7286d946e3e...,06/04/18,09/04/12,11/04/12,1,2,3,1,45,1,2,2,4,3.0,1.0,11,6,D,1,c2eaeada5db1e75552e54382c408fe1c600679060b052c...,F,C,e29c9c180c6279b0b02abd6a1801c7c04082cf486ec027...,6.516391,train,False,False,4,12,16,-2188,2
116,8e7c40db97f7088ea2249bb9da85b1cfce31c8b537dcfa...,06/04/18,14/04/12,15/04/12,1,3,4,0,45,1,2,2,2,3.0,1.0,11,4,G,1,48a9cdd22b472fe4aa225acd45acdbe246de12fade4a59...,F,C,e29c9c180c6279b0b02abd6a1801c7c04082cf486ec027...,8.819591,train,False,False,4,8,8,-2183,1
129,085264337c2af42038a81b4ec4d9e278d9699e5662d404...,06/04/18,19/04/12,21/04/12,3,1,2,0,45,3,3,4,1,2.0,5.0,5,3,B,1,914d3756e6da218a071578d46a3a3c4fce3b773a1736cb...,F,C,39fa9ec190eee7b6f4dff1100d6343e10918d044c75eac...,8.246436,train,False,False,2,2,2,-2178,2
130,7294817a3becfb22650a6e4cf0dd7d2a3a1a3969e1e01d...,06/04/18,19/04/12,21/04/12,3,1,2,0,45,3,3,4,1,2.0,5.0,5,3,B,1,914d3756e6da218a071578d46a3a3c4fce3b773a1736cb...,F,C,39fa9ec190eee7b6f4dff1100d6343e10918d044c75eac...,3.193578,train,False,False,2,2,2,-2178,2
147,e36df886dd8222cad63291754719419d5ff50b44e525e0...,06/04/18,22/04/12,26/04/12,3,3,4,0,45,1,1,3,1,3.0,9.0,6,3,B,2,0beacf60d9f72cdfe3f4b8f1f439c605b9bc37ca3a3922...,F,A,b17ef6d19c7a5b1ee83b907c595526dcb1eb06db8227d6...,7.017013,train,False,False,4,4,4,-2175,4


In [22]:
## it seems like there may be an error in these rows as booking date in in 2018 while stays in 2012. 
## we have 14 such rows. lets replace those by 0 anyway as those will not impact very much
#data[data['days_advance']<0]['days_advance'] = 0
# data['days_advance'][data['days_advance'] < 0] = 0
data.days_advance=data.days_advance.mask(data.days_advance.lt(0),0)

In [23]:
## create a column overstay to represent if the actual stay (based on checkin and check out date) is greater or less or equal to 
## given in the data
conditions = [
    (data['actual_stay'] > data['roomnights']),
    (data['actual_stay'] < data['roomnights']),
    (data['actual_stay'] == data['roomnights'])]
choices = ['overstay', 'understay', 'rightstay']
data['over_stay'] = np.select(conditions, choices)

In [24]:
## we see that advance booking has been done prior to 177 days max. lets group these days to form a different column.
## categorising them in terms of group for eg. groupA has days_advance of 0 days, B has days advance booking from 0 to 7 days
bins = [-0.01, 0.01, 7, 15, 30, 60, 90, 120, 180, 210]
names = ['groupA', 'groupB', 'groupC', 'groupD', 'groupE', 'groupF','groupG', 'groupH', 'groupI']
data['advance_booking_group'] = pd.cut(data['days_advance'], bins, labels=names)

In [25]:
## lets see the data once more
data.head()

,reservation_id,booking_date,checkin_date,checkout_date,channel_code,main_product_code,numberofadults,numberofchildren,persontravellingid,resort_region_code,resort_type_code,room_type_booked_code,roomnights,season_holidayed_code,state_code_residence,state_code_resort,total_pax,member_age_buckets,booking_type_code,memberid,cluster_code,reservationstatusid_code,resort_id,amount_spent_per_room_night_scaled,type,season_holidayed_code_na,state_code_residence_na,total_people,total_adult_night,total_people_night,days_advance,actual_stay,over_stay,advance_booking_group
0,07659f3758d8aee27f5a7e2887adeacb67021cb95ada1b...,05/04/18,05/04/18,06/04/18,3,1,2,0,46,3,3,3,1,2.0,7.0,3,3,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,F,C,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,7.706428,train,False,False,2,2,2,0,1,rightstay,groupA
1,03930f033646d073462b35d411616323597715ac4fc398...,23/01/15,11/04/15,16/04/15,1,1,2,0,46,3,3,4,5,2.0,7.0,5,2,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,F,A,39fa9ec190eee7b6f4dff1100d6343e10918d044c75eac...,6.662563,train,False,False,2,10,10,78,5,rightstay,groupF
2,d145a32920e6587ad95bfe299d80c0affa268220535aaf...,28/01/15,01/02/15,05/02/15,1,1,2,0,47,1,5,4,4,2.0,7.0,1,2,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,E,A,535fa30d7e25dd8a49f1536779734ec8286108d115da50...,7.871602,train,False,False,2,8,8,4,4,rightstay,groupB
3,cfd77f44811ed62f25a220b53324cdbafc662a4c9e5f04...,02/05/15,11/06/15,16/06/15,1,1,2,2,46,2,2,3,5,2.0,7.0,2,2,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,D,A,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,5.344943,train,False,False,4,10,20,40,5,rightstay,groupE
4,937cff9e4dcfc2459620153dfc8b9962ac22bea67dfb29...,02/09/15,14/12/15,19/12/15,1,1,2,0,46,2,2,4,5,2.0,7.0,2,2,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,D,A,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,7.059346,train,False,False,2,10,10,103,5,rightstay,groupG


In [26]:
data.shape, train.shape[0]+test.shape[0]

((488189, 34), 488189)

In [27]:
## add date information columns for each datetype. Although all are not essential but add them anyway
add_datepart(data, 'booking_date')
add_datepart(data, 'checkin_date')
add_datepart(data, 'checkout_date')

In [28]:
## label encode the object columns
## we are not encoding reservationn id as these will not be present in the model building
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for col in [ 'member_age_buckets', 'cluster_code', 'reservationstatusid_code','over_stay','season_holidayed_code_na',
           'state_code_residence_na','memberid','resort_id', 'advance_booking_group']:
    data[col] = le.fit_transform(data[col])

In [29]:
## convert these below object columns to category as model  handles this type much better
# for col in [ 'member_age_buckets', 'cluster_code', 'reservationstatusid_code','over_stay','season_holidayed_code_na',
#            'state_code_residence_na','memberid','resort_id']:
#     data[col] = data[col].astype('category')
# data['advance_booking_group'] = le.fit_transform(data['advance_booking_group'])

In [30]:
data.shape

(488189, 70)

In [31]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 488189 entries, 0 to 488188
Data columns (total 70 columns):
reservation_id                        488189 non-null object
channel_code                          488189 non-null int64
main_product_code                     488189 non-null int64
numberofadults                        488189 non-null int64
numberofchildren                      488189 non-null int64
persontravellingid                    488189 non-null int64
resort_region_code                    488189 non-null int64
resort_type_code                      488189 non-null int64
room_type_booked_code                 488189 non-null int64
roomnights                            488189 non-null int64
season_holidayed_code                 488189 non-null float64
state_code_residence                  488189 non-null float64
state_code_resort                     488189 non-null int64
total_pax                             488189 non-null int64
member_age_buckets                    488189

In [32]:
## everything looks good. only 2 object columns which we ought to remove them next

In [33]:
## split the data to train and test set based on the type level
## define y (target) column
train = data[data['type'] == 'train']
test = data[data['type'] == 'test']
y = train['amount_spent_per_room_night_scaled']

## delete the  id columns which are not part of the model
train_id = train.reservation_id
test_id = test.reservation_id
train = train.drop(['type','amount_spent_per_room_night_scaled','reservation_id'], axis = 1)
# train = train.drop(['type','reservation_id'], axis = 1)
test = test.drop(['type','amount_spent_per_room_night_scaled','reservation_id'], axis = 1)


In [34]:
train.shape, y.shape,test.shape, train.shape[0]+test.shape[0]

((341424, 67), (341424,), (146765, 67), 488189)

### Modelling approach

In [35]:
## split the dataset to train and valid.
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(train, y, test_size=0.25, random_state=2)

In [36]:
max(y), min(y)

(10.81665194, 1.6003971680000002)

In [37]:
## define rmse as per problem statement
def rmse(x,y): return math.sqrt(((x-y)**2).mean()) * 100

#### we will build K fold models

- let's see first LGBM regression model

In [38]:
from lightgbm import LGBMRegressor
# lgb = LGBMRegressor(random_state=1, n_jobs=-1,n_estimators=300, num_leaves=15, learning_rate=0.1,min_child_samples=3, 
#                     reg_alpha=0.0, reg_lambda=0.0, importance_type='gain')
# lgb.fit(X_train, y_train)
# y_valid_pred = lgb.predict(X_valid)
# rmse(y_valid, y_valid_pred)

In [39]:
##97.82
# feature_imp = pd.DataFrame(sorted(zip(lgb.feature_importances_,X_train.columns)), columns=['Value','Feature'])
# feature_imp.sort_values('Value',ascending=False)[0:10]

In [40]:
# to_keep = feature_imp[feature_imp.Value>100].Feature
# df_keep = train[to_keep].copy()
# X_train, X_valid, y_train, y_valid = train_test_split(df_keep, y, test_size=0.25, random_state=2)

- after searching for right params manually we will now build a k-fold model

In [41]:
from lightgbm import LGBMRegressor
from sklearn.model_selection import StratifiedKFold, KFold
## k-fold model define
def k_folds_lgb(X, y, X_test, k,n_est,num_leaves,lr):
    folds = KFold( n_splits = k, shuffle=True, random_state=2)
    y_test = 0
    y_oof = np.zeros((X.shape[0]))
    score = 0
    for i, (train_idx, val_idx) in  enumerate(folds.split(X, y)):
        clf =  LGBMRegressor(random_state =1, n_estimators = n_est, n_jobs = -1,learning_rate=lr, num_leaves=num_leaves,reg_alpha=0.0, 
                             reg_lambda=0.0, min_child_samples=3, categorical_feature = [13,15,16,17,18,19,20,26,27])
        clf.fit(X.iloc[train_idx], y[train_idx])
        y_oof[val_idx] = np.clip(clf.predict(X.iloc[val_idx]),0,20)
        y_test += np.clip(clf.predict(X_test),0,20) / folds.n_splits
        score += rmse(np.clip(clf.predict(X.iloc[val_idx]),0,20), y[val_idx])
        print('Fold: {} score: {}'.format(i,rmse(np.clip(clf.predict(X.iloc[val_idx]),0,20), y[val_idx])))
    print('Avg 100* RMSE', score / folds.n_splits) 
        
    return y_oof, y_test 

In [42]:
y_oof, y_test_lgb = k_folds_lgb(train, y, test, k= 25,n_est=300,num_leaves = 25, lr = 0.20)
min(y_test_lgb), max(y_test_lgb)

/opt/conda/lib/python3.6/site-packages/lightgbm/basic.py:741: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  .format(key))


Fold: 0 score: 100.37199185868151
Fold: 1 score: 97.73281103621349
Fold: 2 score: 98.16387726306296
Fold: 3 score: 96.8837736613688
Fold: 4 score: 97.55234765823144
Fold: 5 score: 96.47956520807632
Fold: 6 score: 98.82921375765571
Fold: 7 score: 98.74921850565809
Fold: 8 score: 98.95262414631773
Fold: 9 score: 98.13983204563762
Fold: 10 score: 96.96448003939526
Fold: 11 score: 97.59491130787315
Fold: 12 score: 98.60666009098813
Fold: 13 score: 98.10164805698479
Fold: 14 score: 98.89357551481423
Fold: 15 score: 98.0369320744463
Fold: 16 score: 99.03549137770182
Fold: 17 score: 96.83066275488996
Fold: 18 score: 97.76409138271441
Fold: 19 score: 98.59975050959062
Fold: 20 score: 97.29137571325171
Fold: 21 score: 96.99627626497562
Fold: 22 score: 98.90348939730077
Fold: 23 score: 97.82307579265121
Fold: 24 score: 96.0178337392395
Avg 100* RMSE 97.97262036630883


(5.890845469106488, 9.769020366820852)

In [43]:
#lowest #Avg RMSE 97.72307084242247 for k=50,lr=0.1,num_leaves=15,n_est=300,reg_alpha=0.1,min_chid_sample=3
submit = pd.DataFrame({'reservation_id':test_id, 'amount_spent_per_room_night_scaled':y_test_lgb}, columns=['reservation_id', 'amount_spent_per_room_night_scaled'])
#submit['num_orders'][submit['amount_spent_per_room_night_scaled'] <0] = 0
submit.to_csv('submit_klgb.csv', index=False)
create_download_link('submit_klgb.csv')

In [44]:
submit.head()

,reservation_id,amount_spent_per_room_night_scaled
341424,7dae1ce6bc8f69481328f2be5c4943077dad5598b5f66d...,8.228024
341425,fe0d4e444e1818436c88f72f1cf800536c2f785e59baeb...,7.387831
341426,540bd4285ad8168e8388f84ee74a82cd4f97dc0a404d3e...,7.584695
341427,09593c907ae262e46f655b4db9e14f54a19eadcfdd2679...,7.891554
341428,f4c50caac68051faf37551d70bb17eebef2a20e2244cb1...,7.699852


- Time for the Gradient boosting regressor model

In [45]:
from sklearn.ensemble import GradientBoostingRegressor
# gbm = GradientBoostingRegressor(random_state =1, n_estimators = 50,learning_rate=0.25,max_depth=5)                              
# gbm.fit(X_train, y_train)
# y_valid_pred = gbm.predict(X_valid)
# rmse(y_valid_pred, y_valid)
# ##98.2395181080143

In [46]:
## k-fold model define
def k_folds_gbm(X, y, X_test, k,n_est, lr):
    folds = KFold( n_splits = k, shuffle=True, random_state=2)
    y_test = 0
    y_oof = np.zeros((X.shape[0]))
    score = 0
    for i, (train_idx, val_idx) in  enumerate(folds.split(X, y)):
        clf =  GradientBoostingRegressor(n_estimators = n_est, learning_rate = lr, random_state=1, max_depth=5)
        clf.fit(X.iloc[train_idx], y[train_idx])
        y_oof[val_idx] = np.clip(clf.predict(X.iloc[val_idx]),0,20)
        y_test += np.clip(clf.predict(X_test),0,20) / folds.n_splits
        score += rmse(np.clip(clf.predict(X.iloc[val_idx]),0,20), y[val_idx])
        print('Fold: {} score: {}'.format(i,rmse(np.clip(clf.predict(X.iloc[val_idx]),0,20), y[val_idx])))
    print('Avg 100* RMSE', score / folds.n_splits) 
        
    return y_oof, y_test 

In [47]:
y_oof, y_test_gbm = k_folds_gbm(train, y, test, k= 25,n_est=150, lr = 0.25)
min(y_test_gbm), max(y_test_gbm)

Fold: 0 score: 100.5563066769934
Fold: 1 score: 97.58914043178764
Fold: 2 score: 98.64801384309168
Fold: 3 score: 97.0420218538109
Fold: 4 score: 97.74056750398636
Fold: 5 score: 96.4531253894402
Fold: 6 score: 99.0220501906504
Fold: 7 score: 98.84127184708326
Fold: 8 score: 99.03807330786051
Fold: 9 score: 97.99643394906285
Fold: 10 score: 97.06506326633928
Fold: 11 score: 97.69071350053555
Fold: 12 score: 98.98271178452606
Fold: 13 score: 98.20472877511162
Fold: 14 score: 98.89488664710187
Fold: 15 score: 98.22568568134497
Fold: 16 score: 98.92583995684454
Fold: 17 score: 97.01906077685682
Fold: 18 score: 97.86566093860657
Fold: 19 score: 98.6548494384981
Fold: 20 score: 97.42646163030926
Fold: 21 score: 97.27579191969913
Fold: 22 score: 99.13722656385868
Fold: 23 score: 98.39422742930395
Fold: 24 score: 96.0041734947946
Avg 100* RMSE 98.10776347189994


(5.423148725557843, 10.258209308675024)

In [48]:
submit = pd.DataFrame({'reservation_id':test_id, 'amount_spent_per_room_night_scaled':y_test_gbm}, columns=['reservation_id', 'amount_spent_per_room_night_scaled'])
#submit['num_orders'][submit['amount_spent_per_room_night_scaled'] <0] = 0
submit.to_csv('submit_kgbm.csv', index=False)
create_download_link('submit_kgbm.csv')

In [49]:
submit.head()

,reservation_id,amount_spent_per_room_night_scaled
341424,7dae1ce6bc8f69481328f2be5c4943077dad5598b5f66d...,8.098145
341425,fe0d4e444e1818436c88f72f1cf800536c2f785e59baeb...,7.389895
341426,540bd4285ad8168e8388f84ee74a82cd4f97dc0a404d3e...,7.578205
341427,09593c907ae262e46f655b4db9e14f54a19eadcfdd2679...,7.879432
341428,f4c50caac68051faf37551d70bb17eebef2a20e2244cb1...,7.727733


- Now lets build XGB regressor model

In [50]:
from xgboost import XGBRegressor
# xgb = XGBRegressor(n_estimators = 50, n_jobs = -1,learning_rate = 0.1, gamma = 0.00, random_state=1,
#                            colsample_bytree=1, max_depth=7)
# xgb.fit(X_train, y_train)
# y_valid_pred = xgb.predict(X_valid)
# rmse(y_valid_pred, y_valid)
# ### 98.52735932116437

In [51]:
from xgboost import XGBRegressor
## k-fold model define
def k_folds_xgb(X, y, X_test, k,n_est, lr):
    folds = KFold( n_splits = k, shuffle=True, random_state=2)
    y_test = 0
    y_oof = np.zeros((X.shape[0]))
    score = 0
    for i, (train_idx, val_idx) in  enumerate(folds.split(X, y)):
        clf =  XGBRegressor(n_estimators = n_est, n_jobs = -1,learning_rate = lr, gamma = 0.00, random_state=1,
                           colsample_bytree=1, max_depth=7)
        clf.fit(X.iloc[train_idx], y[train_idx])
        y_oof[val_idx] = np.clip(clf.predict(X.iloc[val_idx]),0,20)
        y_test += np.clip(clf.predict(X_test),0,20) / folds.n_splits
        score += rmse(np.clip(clf.predict(X.iloc[val_idx]),0,20), y[val_idx])
        print('Fold: {} score: {}'.format(i,rmse(np.clip(clf.predict(X.iloc[val_idx]),0,20), y[val_idx])))
    print('Avg 100* RMSE', score / folds.n_splits) 
        
    return y_oof, y_test 

In [52]:
y_oof, y_test_xgb = k_folds_xgb(train, y, test, k= 25,n_est=200, lr = 0.10)
min(y_test_xgb), max(y_test_xgb)

/opt/conda/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


Fold: 0 score: 100.62203314815612
Fold: 1 score: 97.56347683452898
Fold: 2 score: 98.35315220152268
Fold: 3 score: 96.95469386393448
Fold: 4 score: 97.51159567820865
Fold: 5 score: 96.36815176028234
Fold: 6 score: 98.92386866061506
Fold: 7 score: 98.74810539664796
Fold: 8 score: 99.05414035978998
Fold: 9 score: 97.83149939327775
Fold: 10 score: 96.97861452989493
Fold: 11 score: 97.72338125428556
Fold: 12 score: 98.86670361092264
Fold: 13 score: 98.14643214981935
Fold: 14 score: 98.85346314020269
Fold: 15 score: 98.03794386240952
Fold: 16 score: 98.76502566208289
Fold: 17 score: 97.09426942203943
Fold: 18 score: 97.74517338599429
Fold: 19 score: 98.53254670101302
Fold: 20 score: 97.25904111185923
Fold: 21 score: 97.17655705579858
Fold: 22 score: 98.91274682738982
Fold: 23 score: 98.12357995261482
Fold: 24 score: 95.85277355800021
Avg 100* RMSE 97.99995878085164


(5.9378695, 10.024941)

In [53]:
submit = pd.DataFrame({'reservation_id':test_id, 'amount_spent_per_room_night_scaled':y_test_xgb}, columns=['reservation_id', 'amount_spent_per_room_night_scaled'])
#submit['num_orders'][submit['amount_spent_per_room_night_scaled'] <0] = 0
submit.to_csv('submit_kxgb.csv', index=False)
create_download_link('submit_kxgb.csv')

In [54]:
submit.head()

,reservation_id,amount_spent_per_room_night_scaled
341424,7dae1ce6bc8f69481328f2be5c4943077dad5598b5f66d...,8.133066
341425,fe0d4e444e1818436c88f72f1cf800536c2f785e59baeb...,7.386870
341426,540bd4285ad8168e8388f84ee74a82cd4f97dc0a404d3e...,7.588923
341427,09593c907ae262e46f655b4db9e14f54a19eadcfdd2679...,7.911251
341428,f4c50caac68051faf37551d70bb17eebef2a20e2244cb1...,7.757587


- finally lets do random forest model

In [55]:
from sklearn.ensemble import RandomForestRegressor
# rf = RandomForestRegressor(random_state =1, n_estimators = 50, n_jobs = -1,min_samples_leaf=15,max_features = 'sqrt')
# rf.fit(X_train, y_train)
# y_valid_pred = rf.predict(X_valid)
# rmse(y_valid_pred, y_valid)
### 99.29315280701118

In [56]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import StratifiedKFold, KFold
## k-fold model define
def k_folds_rf(X, y, X_test, k,n_est,num_leaves,max_feat):
    folds = KFold( n_splits = k, shuffle=True, random_state=2)
    y_test = 0
    y_oof = np.zeros((X.shape[0]))
    score = 0
    for i, (train_idx, val_idx) in  enumerate(folds.split(X, y)):
        clf =  RandomForestRegressor(random_state =1, n_estimators = n_est, n_jobs = -1,min_samples_leaf=num_leaves,max_features = max_feat)
        clf.fit(X.iloc[train_idx], y[train_idx])
        y_oof[val_idx] = clf.predict(X.iloc[val_idx])
        y_test += clf.predict(X_test) / folds.n_splits
        score += rmse(clf.predict(X.iloc[val_idx]), y[val_idx])
        print('Fold: {} score: {}'.format(i,rmse(clf.predict(X.iloc[val_idx]), y[val_idx])))
    print('Avg 100* RMSE', score / folds.n_splits) 
        
    return y_oof, y_test 

In [57]:
y_oof, y_test_rf = k_folds_rf(train, y, test, k= 25,n_est=150,num_leaves = 15, max_feat ='sqrt')
min(y_test_rf), max(y_test_rf)

Fold: 0 score: 101.84682319005974
Fold: 1 score: 98.6112658416595
Fold: 2 score: 99.37520144361478
Fold: 3 score: 98.14468291022564
Fold: 4 score: 98.62847093608778
Fold: 5 score: 97.41258175022445
Fold: 6 score: 100.06666887426346
Fold: 7 score: 99.94895583922386
Fold: 8 score: 99.86919896402313
Fold: 9 score: 99.09372160779817
Fold: 10 score: 98.37014277930976
Fold: 11 score: 98.64211176285657
Fold: 12 score: 99.72325809496475
Fold: 13 score: 98.98597648637046
Fold: 14 score: 100.10327191968489
Fold: 15 score: 99.11693133688577
Fold: 16 score: 99.60202574837518
Fold: 17 score: 98.04279152515338
Fold: 18 score: 98.94927844462153
Fold: 19 score: 99.57391527639467
Fold: 20 score: 98.30855975565898
Fold: 21 score: 97.94130118676509
Fold: 22 score: 100.2073021633416
Fold: 23 score: 99.57883925494878
Fold: 24 score: 96.72797044166236
Avg 100* RMSE 99.07484990136696


(6.454266495147902, 9.167653906813483)

In [58]:
##avg score 99.16587677246167
submit = pd.DataFrame({'reservation_id':test_id, 'amount_spent_per_room_night_scaled':y_test_rf}, columns=['reservation_id', 'amount_spent_per_room_night_scaled'])
#submit['num_orders'][submit['amount_spent_per_room_night_scaled'] <0] = 0
submit.to_csv('submit_krf.csv', index=False)
create_download_link('submit_krf.csv')

In [59]:
submit.head()

,reservation_id,amount_spent_per_room_night_scaled
341424,7dae1ce6bc8f69481328f2be5c4943077dad5598b5f66d...,8.007752
341425,fe0d4e444e1818436c88f72f1cf800536c2f785e59baeb...,7.554216
341426,540bd4285ad8168e8388f84ee74a82cd4f97dc0a404d3e...,7.636905
341427,09593c907ae262e46f655b4db9e14f54a19eadcfdd2679...,7.817560
341428,f4c50caac68051faf37551d70bb17eebef2a20e2244cb1...,7.799081


- averaging the test set predictions for these 4 models. we will assign weights based on k-fold validation score

In [60]:
## LGBM performed better, so giving modre weights
y_test_avg = 0.27 * y_test_lgb + 0.24 * y_test_xgb + 0.25 * y_test_gbm + 0.24 * y_test_rf
submit = pd.DataFrame({'reservation_id':test_id, 'amount_spent_per_room_night_scaled':y_test_avg}, columns=['reservation_id', 'amount_spent_per_room_night_scaled'])
#submit['num_orders'][submit['amount_spent_per_room_night_scaled'] <0] = 0
submit.to_csv('submit_avg.csv', index=False)
create_download_link('submit_avg.csv')

In [61]:
submit.head()

,reservation_id,amount_spent_per_room_night_scaled
341424,7dae1ce6bc8f69481328f2be5c4943077dad5598b5f66d...,8.119899
341425,fe0d4e444e1818436c88f72f1cf800536c2f785e59baeb...,7.428049
341426,540bd4285ad8168e8388f84ee74a82cd4f97dc0a404d3e...,7.596618
341427,09593c907ae262e46f655b4db9e14f54a19eadcfdd2679...,7.875492
341428,f4c50caac68051faf37551d70bb17eebef2a20e2244cb1...,7.744494


#### deep learning approach

- i will use fastai tabular learner datablock api